1. Load the data
    - Check the head, tail, shape, info, describe
2. Clean the data
    - Missing value %
    - Numerical Columns (outlier treatment)
    - Categorical Columns (columns with very high proportion of same value)
3. Divide the columns into 3 categories
    - Numerical 
    - Categorical
    - Date Time


# 1. Loading dependencies & datasets

Lets start by loading our dependencies. We can keep adding any imports to this cell block, as we write mode and mode code.

In [1710]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

warnings.filterwarnings('ignore')
%matplotlib inline
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [1711]:
telecom_df = pd.read_csv("../../datasets/telecom-churn/train.csv")
unseen_telecom_df = pd.read_csv("../../datasets/telecom-churn/test.csv")
sample_df = pd.read_csv("../../datasets/telecom-churn/sample.csv")
data_dict_df = pd.read_csv("../../datasets/telecom-churn/data_dictionary.csv")

print(telecom_df.shape)
print(unseen_telecom_df.shape)
print(sample_df.shape)
print(data_dict_df.shape)

(69999, 172)
(30000, 171)
(30000, 2)
(36, 2)


In [1712]:
telecom_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69999 entries, 0 to 69998
Data columns (total 172 columns):
 #    Column                    Non-Null Count  Dtype  
---   ------                    --------------  -----  
 0    id                        69999 non-null  int64  
 1    circle_id                 69999 non-null  int64  
 2    loc_og_t2o_mou            69297 non-null  float64
 3    std_og_t2o_mou            69297 non-null  float64
 4    loc_ic_t2o_mou            69297 non-null  float64
 5    last_date_of_month_6      69999 non-null  object 
 6    last_date_of_month_7      69600 non-null  object 
 7    last_date_of_month_8      69266 non-null  object 
 8    arpu_6                    69999 non-null  float64
 9    arpu_7                    69999 non-null  float64
 10   arpu_8                    69999 non-null  float64
 11   onnet_mou_6               67231 non-null  float64
 12   onnet_mou_7               67312 non-null  float64
 13   onnet_mou_8               66296 non-null  fl

In [1713]:
print('Number of rows are: ', telecom_df.shape[0])
print('Number of cols are: ', telecom_df.shape[1])
print('Number of duplicated rows are: ', telecom_df.duplicated().sum())

Number of rows are:  69999
Number of cols are:  172
Number of duplicated rows are:  0


#### Dropping columns with all NULL Values

In [1714]:
all_null_value_cols = [col for col in telecom_df.columns if telecom_df[col].isnull().all() == True]
unseen_all_null_value_cols = [col for col in unseen_telecom_df.columns if unseen_telecom_df[col].isnull().all() == True]

print('Columns with all null values are:', all_null_value_cols)
print('Number of columns with all null values are: ', len(all_null_value_cols))

print('Columns with all null values in unseen data are:', unseen_all_null_value_cols)
print('Number of columns with all null values in unseen data are: ', len(unseen_all_null_value_cols))

Columns with all null values are: []
Number of columns with all null values are:  0
Columns with all null values in unseen data are: []
Number of columns with all null values in unseen data are:  0


##### Dropping columns with SINGLE Values

In [1715]:
single_value_cols = []
for col in telecom_df.columns:
    if telecom_df[col].nunique() == 1:
        single_value_cols.append(col)
print('Columns with single value: ', single_value_cols)
print('Number of columns with sigle values are: ', len(single_value_cols))
telecom_df = telecom_df.drop(telecom_df[single_value_cols], axis = 1)
unseen_telecom_df = unseen_telecom_df.drop(unseen_telecom_df[single_value_cols], axis = 1)
print('Dropped all the columns with single values in available data. Shape now is: ', telecom_df.shape)
print('Dropped all the columns with single values in unseen data. Shape now is: ', unseen_telecom_df.shape)

Columns with single value:  ['circle_id', 'loc_og_t2o_mou', 'std_og_t2o_mou', 'loc_ic_t2o_mou', 'last_date_of_month_6', 'last_date_of_month_7', 'last_date_of_month_8', 'std_og_t2c_mou_6', 'std_og_t2c_mou_7', 'std_og_t2c_mou_8', 'std_ic_t2o_mou_6', 'std_ic_t2o_mou_7', 'std_ic_t2o_mou_8']
Number of columns with sigle values are:  13
Dropped all the columns with single values in available data. Shape now is:  (69999, 159)
Dropped all the columns with single values in unseen data. Shape now is:  (30000, 158)


#### Dropping the id column

In [1716]:
telecom_df = telecom_df.drop(columns=['id'])
print('Dropped the id column in available data. Shape now is: ',telecom_df.shape)

Dropped the id column in available data. Shape now is:  (69999, 158)


#### Null Value Treatments

In [1717]:
def calculate_null_values(telecom_df):
    null_percentages = (telecom_df.isnull().sum() / telecom_df.shape[0]) * 100
    null_values_df = pd.DataFrame({'Column Name': null_percentages.index, 'Null Percentage': null_percentages})
    null_values_df = null_values_df.loc[null_values_df['Null Percentage'] != 0.00]
    return null_values_df

In [1718]:
null_values_df = calculate_null_values(telecom_df)
unseen_null_values_df = calculate_null_values(unseen_telecom_df)
print('Number of columns with NON-ZERO null values in available data: ',null_values_df.shape)
print('Number of columns with NON-ZERO null values in unseen data: ',unseen_null_values_df.shape)

Number of columns with NON-ZERO null values in available data:  (114, 2)
Number of columns with NON-ZERO null values in unseen data:  (114, 2)


In [1719]:
print('Columns with Null Percentage greater than 70.00')
null_values_df[null_values_df['Null Percentage'] > 70.00]

Columns with Null Percentage greater than 70.00


,Column Name,Null Percentage
date_of_last_rech_data_6,date_of_last_rech_data_6,74.902499
date_of_last_rech_data_7,date_of_last_rech_data_7,74.478207
date_of_last_rech_data_8,date_of_last_rech_data_8,73.689624
total_rech_data_6,total_rech_data_6,74.902499
total_rech_data_7,total_rech_data_7,74.478207
total_rech_data_8,total_rech_data_8,73.689624
max_rech_data_6,max_rech_data_6,74.902499
max_rech_data_7,max_rech_data_7,74.478207
max_rech_data_8,max_rech_data_8,73.689624
count_rech_2g_6,count_rech_2g_6,74.902499


In [1720]:
print('Imputing the categorical columns with appropriate values')
null_cat_columns = [
    'night_pck_user_6',
    'night_pck_user_7',
    'night_pck_user_8',
    'fb_user_6',
    'fb_user_7',
    'fb_user_8',
]
telecom_df[null_cat_columns] = telecom_df[null_cat_columns].fillna(-1).astype('category')
unseen_telecom_df[null_cat_columns] = unseen_telecom_df[null_cat_columns].fillna(-1).astype('category')
null_values_df = calculate_null_values(telecom_df)
unseen_null_values_df = calculate_null_values(unseen_telecom_df)
print('Number of columns with NON-ZERO null values in available data: ',null_values_df.shape)
print('Number of columns with NON-ZERO null values in unseen data: ',unseen_null_values_df.shape)

Imputing the categorical columns with appropriate values
Number of columns with NON-ZERO null values in available data:  (108, 2)
Number of columns with NON-ZERO null values in unseen data:  (108, 2)


In [1721]:
print('Removing the date columns with null values as they are not useful')
null_date_columns = [
    'date_of_last_rech_data_6',
    'date_of_last_rech_data_7',
    'date_of_last_rech_data_8',
    'date_of_last_rech_6',
    'date_of_last_rech_7',
    'date_of_last_rech_8'
]
telecom_df = telecom_df.drop(null_date_columns, axis = 1)
unseen_telecom_df = unseen_telecom_df.drop(null_date_columns, axis = 1)
null_values_df = calculate_null_values(telecom_df)
unseen_null_values_df = calculate_null_values(unseen_telecom_df)
print('Number of columns with NON-ZERO null values in available data: ',null_values_df.shape)
print('Number of columns with NON-ZERO null values in unseen data: ',unseen_null_values_df.shape)

Removing the date columns with null values as they are not useful
Number of columns with NON-ZERO null values in available data:  (102, 2)
Number of columns with NON-ZERO null values in unseen data:  (102, 2)


In [1722]:
print('Imputing the numerical columns with null values')
null_num_columns = null_values_df[null_values_df['Null Percentage'] > 70.00]['Column Name'].tolist()
telecom_df[null_num_columns] = telecom_df[null_num_columns].fillna(0.0)
unseen_telecom_df[null_num_columns] = unseen_telecom_df[null_num_columns].fillna(0.0)
null_values_df = calculate_null_values(telecom_df)
unseen_null_values_df = calculate_null_values(unseen_telecom_df)
print('Number of columns with NON-ZERO null values in available data: ',null_values_df.shape)
print('Number of columns with NON-ZERO null values in unseen data: ',unseen_null_values_df.shape)

Imputing the numerical columns with null values
Number of columns with NON-ZERO null values in available data:  (81, 2)
Number of columns with NON-ZERO null values in unseen data:  (81, 2)


In [1723]:
print('Imputing the columns with NAN values using Simple Imputer')
imputer = SimpleImputer(missing_values = np.nan, strategy='mean')
#imputer = KNNImputer(n_neighbors = 5)
telecom_df[null_values_df['Column Name']] = imputer.fit_transform(telecom_df[null_values_df['Column Name']])
unseen_telecom_df[unseen_null_values_df['Column Name']] = imputer.fit_transform(unseen_telecom_df[unseen_null_values_df['Column Name']])  

null_values_df = calculate_null_values(telecom_df)
unseen_null_values_df = calculate_null_values(unseen_telecom_df)

print('Number of columns with NON-ZERO null values in available data: ',null_values_df.shape)
print('Number of columns with NON-ZERO null values in unseen data: ',unseen_null_values_df.shape)

Imputing the columns with NAN values using Simple Imputer
Number of columns with NON-ZERO null values in available data:  (0, 2)
Number of columns with NON-ZERO null values in unseen data:  (0, 2)


In [1724]:
print('Deriving total value columns for available data:')
telecom_df['total_rech_data_amt_6'] = telecom_df['total_rech_data_6'] * telecom_df['av_rech_amt_data_6']
telecom_df['total_rech_data_amt_7'] = telecom_df['total_rech_data_7'] * telecom_df['av_rech_amt_data_7']
telecom_df['total_rech_data_amt_8'] = telecom_df['total_rech_data_8'] * telecom_df['av_rech_amt_data_8']

telecom_df['total_amt_6'] = telecom_df['total_rech_amt_6'] + telecom_df['total_rech_data_amt_6']
telecom_df['total_amt_7'] = telecom_df['total_rech_amt_7'] + telecom_df['total_rech_data_amt_7']
telecom_df['total_amt_8'] = telecom_df['total_rech_amt_8'] + telecom_df['total_rech_data_amt_8']

telecom_df['total_value'] = telecom_df['total_amt_6'] + telecom_df['total_amt_7']

print('Deriving total value columns for unseen data:')
unseen_telecom_df['total_rech_data_amt_6'] = unseen_telecom_df['total_rech_data_6'] * unseen_telecom_df['av_rech_amt_data_6']
unseen_telecom_df['total_rech_data_amt_7'] = unseen_telecom_df['total_rech_data_7'] * unseen_telecom_df['av_rech_amt_data_7']
unseen_telecom_df['total_rech_data_amt_8'] = unseen_telecom_df['total_rech_data_8'] * unseen_telecom_df['av_rech_amt_data_8']

unseen_telecom_df['total_amt_6'] = unseen_telecom_df['total_rech_amt_6'] + unseen_telecom_df['total_rech_data_amt_6']
unseen_telecom_df['total_amt_7'] = unseen_telecom_df['total_rech_amt_7'] + unseen_telecom_df['total_rech_data_amt_7']
unseen_telecom_df['total_amt_8'] = unseen_telecom_df['total_rech_amt_8'] + unseen_telecom_df['total_rech_data_amt_8']

unseen_telecom_df['total_value'] = unseen_telecom_df['total_amt_6'] + unseen_telecom_df['total_amt_7']

Deriving total value columns for available data:
Deriving total value columns for unseen data:


In [1725]:
cat_dummies_df = pd.get_dummies(telecom_df[null_cat_columns], drop_first = True, dtype = 'int')
unseen_cat_dummies_df = pd.get_dummies(unseen_telecom_df[null_cat_columns], drop_first = True, dtype = 'int')
telecom_df = pd.concat([telecom_df, cat_dummies_df], axis=1)
unseen_telecom_df = pd.concat([unseen_telecom_df, unseen_cat_dummies_df], axis=1)

In [1726]:
high_value_telecom_df = telecom_df[(telecom_df['total_value'] > telecom_df['total_value'].quantile(0.75))].copy()
unseen_high_value_telecom_df = unseen_telecom_df[(unseen_telecom_df['total_value'] > unseen_telecom_df['total_value'].quantile(0.75))].copy()

print('Filtered high value customers from available data. Shape now is: ',high_value_telecom_df.shape)
print('Filtered high value customers from unseen data. Shape now is: ',unseen_high_value_telecom_df.shape)

Filtered high value customers from available data. Shape now is:  (17495, 171)
Filtered high value customers from unseen data. Shape now is:  (7499, 171)


In [1727]:
telecom_df['churn_probability'] = telecom_df['churn_probability'].astype('category')
X = high_value_telecom_df.drop(['churn_probability'], axis=1)
y = high_value_telecom_df['churn_probability']
X_unseen = unseen_high_value_telecom_df
X_unseen_id = X_unseen.pop(('id'))

Splitting train and test data to avoid any contamination of the test data

In [1728]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 100)

In [1729]:
# Checking the shape after split
print('Train X Shape: ', X_train.shape)
print('Test X Shape: ', X_test.shape)
print('Unseen X Shape: ', X_unseen.shape)
print('Train y Shape: ', y_train.shape)
print('Test y Shape: ', y_test.shape)

Train X Shape:  (13996, 170)
Test X Shape:  (3499, 170)
Unseen X Shape:  (7499, 170)
Train y Shape:  (13996,)
Test y Shape:  (3499,)


In [1730]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_unseen_scaled = scaler.transform(X_unseen)

In [1731]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns = X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns = X_test.columns)
X_unseen_scaled = pd.DataFrame(X_unseen_scaled, columns = X_unseen.columns)

In [1732]:
logistic_regression = LogisticRegression()
rfe_selector = RFE(logistic_regression, n_features_to_select = 20)
rfe_selector.fit(X_train_scaled, y_train)
selected_features = X_train_scaled.columns[rfe_selector.support_]

In [1733]:
X_train_scaled_selected = X_train_scaled[selected_features]
X_test_scaled_selected = X_test_scaled[selected_features]
X_unseen_scaled_selected = X_unseen_scaled[selected_features]

In [1734]:
print('Train X Shape: ', X_train_scaled_selected.shape)
print('Test X Shape: ', X_test_scaled_selected.shape)
print('Unseen X Shape: ', X_unseen_scaled_selected.shape)
print('Train y Shape: ', y_train.shape)
print('Test y Shape: ', y_test.shape)

Train X Shape:  (13996, 20)
Test X Shape:  (3499, 20)
Unseen X Shape:  (7499, 20)
Train y Shape:  (13996,)
Test y Shape:  (3499,)


In [1735]:
logistic_regression.fit(X_train_scaled_selected, y_train)
y_train_pred = logistic_regression.predict(X_train_scaled_selected)
train_score = logistic_regression.score(X_train_scaled_selected, y_train)

In [1736]:
y_test_pred = logistic_regression.predict(X_test_scaled_selected)
test_score = logistic_regression.score(X_test_scaled_selected, y_test)

In [1737]:
print("Train accuracy:", round(train_score * 100, ndigits=2))
print("Test accuracy:", round(test_score * 100, ndigits=2))

Train accuracy: 91.91
Test accuracy: 92.37


In [1738]:
y_unseen_pred = logistic_regression.predict(X_unseen_scaled_selected)
y_unseen_pred_df = pd.DataFrame(y_unseen_pred)
y_unseen_pred_df = pd.concat([y_unseen_pred_df, X_unseen_id])
y_unseen_pred_df.head()

,0
0,0
1,0
2,0
3,0
4,0


# 7. Creating submission file

For submission, we need to make sure that the format is exactly the same as the sample.csv file. It contains 2 columns, id and churn_probability

Lets first select the columns that we want to work with (or create them, if you have done any feature engineering)

In [ ]:
submission_data = unseen_telecom_df[selected_features]

(30000, 20)

Next, lets create a new column in the unseen dataset called churn_probability and use the model pipeline to predict the probabilities for this data

In [ ]:
unseen_telecom_df['churn_probability'] = logistic_regression.predict(submission_data)
unseen_telecom_df.head()
# output = unseen_telecom_df[['id','churn_probability']]
# output.head()

,id,churn_probability
0,69999,0
1,70000,0
2,70001,0
3,70002,0
4,70003,0


Finally, lets create a csv file out of this dataset, ensuring to set index=False to avoid an addition column in the csv.

In [ ]:
output.to_csv('submission_lr.csv',index=False)

NameError: name 'output' is not defined

You can now take this file and upload it as a submission on Kaggle.